#### Tạo bảng và thêm data vào ####

In [34]:
DROP TABLE IF EXISTS dim_customers

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 35, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [35]:
DROP TABLE IF EXISTS updated_source

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 36, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [36]:
-- Welcome to your new notebook
-- Type here in the cell editor to add code!
CREATE TABLE IF NOT EXISTS dim_customers(customerID int,name string, address string, current boolean, effectiveDate timestamp,endDate timestamp )

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 37, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [37]:
insert into dim_customers values 
(1,'Mahesh','Bangalore',1,CURRENT_TIMESTAMP(),'9999-12-31'),
(2,'ram','Hyderabad',1,CURRENT_TIMESTAMP(),'9999-12-31'),
(3,'ravy','Chennai',1,CURRENT_TIMESTAMP(),'9999-12-31'),
(4,'raj','Pune',1,CURRENT_TIMESTAMP(),'9999-12-31')d

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 38, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [38]:

CREATE TABLE IF NOT EXISTS updated_source(customerID int,name string, address string, effectiveDate timestamp)

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 39, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [39]:
insert into updated_source values
(5,'Sridhar','Delhi',CURRENT_TIMESTAMP()),
(6,'Prasad','Mumbai',CURRENT_TIMESTAMP()),
(2,'ram','Bangalore',CURRENT_TIMESTAMP()),
(1,'Mahesh','Hyderabad',CURRENT_TIMESTAMP())


StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 40, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

#### Tìm những data đã có thay đổi ở source và bảng dim ####


#### Dựa vào dim_customers thì kết quả bảng dim thay đổi sẽ sai, nên phải dựa vào bảng update để join thêm data bảng dim mới ra kết quả đúng ####

In [40]:
SELECT dim_customers.customerID as mergeKey, dim_customers.*
FROM dim_customers
UNION ALL
SELECT NULL as mergerKey, dim_customers.*
FROM dim_customers JOIN updated_source ON dim_customers.customerID = updated_source.customerID
WHERE dim_customers.current = 1 and dim_customers.address = updated_source.address

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 41, Finished, Available)

<Spark SQL result set with 4 rows and 7 fields>

In [41]:
SELECT dim_customers.customerID as mergeKey, dim_customers.*
FROM dim_customers
UNION ALL
SELECT NULL as mergerKey, dim_customers.*
FROM dim_customers JOIN updated_source ON dim_customers.customerID = updated_source.customerID
WHERE dim_customers.current = 1 and dim_customers.address <> updated_source.address

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 42, Finished, Available)

<Spark SQL result set with 6 rows and 7 fields>

#### Nếu merge data bảng dim ở trên với bảng dim mới cần update data sẽ có kết quả sai, dưới đây mới là cách làm đúng với merge với data đó ####

In [42]:
SELECT updated_source.customerID as mergeKey, updated_source.*
FROM updated_source
UNION ALL
SELECT NULL as mergerKey, updated_source.*
FROM updated_source JOIN dim_customers ON dim_customers.customerID = updated_source.customerID
WHERE dim_customers.current = 1 and dim_customers.address <> updated_source.address

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 43, Finished, Available)

<Spark SQL result set with 6 rows and 5 fields>

#### Điều này có nghĩa là data hai hàng 5 6 vừa mới đc insert vào, data có key 1 2 cũ sẽ bị chuyển thành false và endDate = effectiveDate của data mới import vào.Bằng cách merge data gốc và data vừa union ở trên ####

In [44]:
MERGE INTO dim_customers 
USING (
SELECT updated_source.customerID as mergeKey, updated_source.*
FROM updated_source
UNION ALL
SELECT NULL as mergerKey, updated_source.*
FROM updated_source JOIN dim_customers ON dim_customers.customerID = updated_source.customerID
WHERE dim_customers.current = 1 and dim_customers.address <> updated_source.address
) staged_data
ON dim_customers.customerID = staged_data.mergeKey 

WHEN MATCHED  AND dim_customers.current = 1 AND dim_customers.address <> staged_data.address  
THEN 
    UPDATE SET dim_customers.current = 0, endDate = staged_data.effectiveDate 

WHEN NOT MATCHED THEN 
    INSERT(customerID, name, address, current, effectiveDate, endDate)
    VALUES (staged_data.customerID, staged_data.name, staged_data.address, 1,staged_data.effectiveDate, '9999-12-31');

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 45, Finished, Available)

<Spark SQL result set with 1 rows and 4 fields>

#### Data mới được cập nhật sẽ có status là true, data cũ cùng ID sẽ bị đổi thành false và endDate sẽ được cập nhật ####

In [45]:
select * from dim_customers

StatementMeta(, 6da5cf11-d0b3-49c1-a04b-8555dbdd15be, 46, Finished, Available)

<Spark SQL result set with 8 rows and 6 fields>